In [ ]:
from src.data.data import *
from src.orcml import *
from src.plotting import *
from src.utils.graph_utils import *
from src.isorc import *
from src.utils.embeddings import *
from sklearn.manifold import TSNE
import umap
import numpy as np
import torchvision

%load_ext autoreload

exp_params = {
    'mode': 'nbrs',
    'n_neighbors': 25,
    'epsilon': None,
    'lda': 1e-5,
    'delta': 0.8
}

In [2]:
%autoreload 2
# quadratics
n_points = 2000
noise = 0.175
noise_thresh = None

return_dict = quadratics(n_points=n_points, noise=noise, noise_thresh=noise_thresh, supersample=True)

In [ ]:
orcmanl = ORCManL(
    exp_params=exp_params,
    verbose=True,
    reattach=True,
)
orcmanl.fit(return_dict['data'])
G = orcmanl.G.copy()

In [ ]:
plot_data_2D(return_dict['data'], color=None, title=None, node_size=3)
plt.title('Original Data')

In [ ]:
umap_emb = umap.UMAP(n_neighbors=25, min_dist=0.1, metric='euclidean').fit_transform(return_dict['data'])
plot_graph_2D(umap_emb, G, node_color=return_dict['cluster'][G.nodes()], edge_color=orcmanl.orcs, title=None, edge_width=0.15, node_size=0.0)
plt.title('UMAP Embedding with annotated edge ORC')

In [17]:
%autoreload 2
# quadratics
n_points = 2000
noise = 0.1
noise_thresh = None

return_dict = concentric_circles(n_points=n_points, factor=0.4, noise=noise, noise_thresh=noise_thresh, supersample=True)

In [ ]:
plot_data_2D(return_dict['data'], color=None, title=None, node_size=3)
plt.title('Original Data')

In [ ]:
orcmanl = ORCManL(
    exp_params=exp_params,
    verbose=True,
    reattach=True,
)
orcmanl.fit(return_dict['data'])
G = orcmanl.G.copy()

umap_emb = umap.UMAP(n_neighbors=25, min_dist=0.1, metric='euclidean').fit_transform(return_dict['data'])
plot_graph_2D(umap_emb, G, node_color=return_dict['cluster'][G.nodes()], edge_color=orcmanl.orcs, title=None, edge_width=0.15, node_size=0.0)
plt.title('UMAP Embedding with annotated edge ORC')

In [ ]:

def orc_fa(G, a, b, scaling_ratio=2):
    A_orc = nx.to_numpy_array(G, weight='ricciCurvature')
    A_edge = nx.to_numpy_array(G, weight='weight')
    
    edge_mask = np.where(A_edge > 0, 1, 0)

    assert b >= a, "b must be greater than a"
    # map each row to [a, b]
    max_orc_row = 1
    min_orc_row = -2

    A_orc_normalized = (b - a) / (max_orc_row - min_orc_row) * (A_orc - min_orc_row) + a
    A_orc_normalized = (A_orc_normalized + A_orc_normalized.T) / 2
    A_orc_normalized *= edge_mask

    G_normalized = G.copy()
    node_indices = list(G.nodes())
    for i, (u, v) in enumerate(G_normalized.edges):
        idx_u = node_indices.index(u)
        idx_v = node_indices.index(v)
        assert A_edge[idx_u, idx_v] != 0 or u == v, "index misalignment"
        G_normalized[u][v]['orc_weight'] = A_orc_normalized[idx_u, idx_v]
    
    spectral_init = nx.spectral_layout(G_normalized, weight='weight')
    print('Finished spectral initialization')
    # force directed layout
    node_mass = {node:1 for node in G_normalized.nodes()}
    pos = nx.forceatlas2_layout(G_normalized, pos=spectral_init, seed=42, weight='orc_weight', scaling_ratio=scaling_ratio, node_mass=node_mass)
    print('Finished spring layout')
    return pos, G_normalized

pos, G_normalized = orc_fa(G, 0, 1, scaling_ratio=2)
plot_graph_2D(pos, G_normalized, node_color=return_dict['cluster'][G_normalized.nodes()], edge_color=orcmanl.orcs, title=None, edge_width=0.15, node_size=0.0)